In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
np.random.randint(len(patterns), size=10)

array([1, 0, 0, 1, 0, 0, 1, 0, 0, 1])

In [28]:
list(range(10,7,-1))

[10, 9, 8]

In [2]:
n_samples = 20000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i: list(range(i, i+3)),
    lambda i: list(range(i, i-3, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number in range(3):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        pattern = np.random.randint(len(patterns))
        line = patterns[pattern](i)
        if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)
            
        sample_out.append(line + [60])
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [32]:
sample_inputs[:10]

[[[31, 31, 32, 33], [0, 27, 28, 29], [0, 20, 21, 22]],
 [[16, 16, 17, 18], [0, 42, 41, 40], [0, 30, 31, 32]],
 [[10, 10, 9, 8], [0, 31, 30, 29], [0, 22, 21, 20]],
 [[48, 48, 47, 46], [0, 34, 33, 32], [0, 45, 46, 47]],
 [[23, 23, 22, 21], [0, 13, 12, 11], [0, 41, 40, 39]],
 [[32, 32, 31, 30], [0, 25, 24, 23], [0, 49, 50, 51]],
 [[15, 15, 14, 13], [0, 14, 15, 16], [0, 44, 43, 42]],
 [[16, 16, 15, 14], [0, 41, 40, 39], [0, 22, 23, 24]],
 [[39, 39, 40, 41], [0, 44, 43, 42], [0, 45, 46, 47]],
 [[44, 44, 43, 42], [0, 49, 50, 51], [0, 42, 43, 44]]]

In [3]:
latent_dim = 512

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

model = Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_1 (LSTM)                                                     [(None, None, 512), (None, 512), (None, 512 1175552                 input_1[0][0]                                                 

In [4]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=32, epochs=20)

Epoch 1/20
20000/20000 [==============================] - 22s 1ms/step - loss: 4.6532 - dense_1_loss: 1.2867 - dense_2_loss: 1.7137 - dense_3_loss: 1.6528
Epoch 2/20
20000/20000 [==============================] - 19s 931us/step - loss: 2.5231 - dense_1_loss: 0.2864 - dense_2_loss: 1.1208 - dense_3_loss: 1.1158
Epoch 3/20
20000/20000 [==============================] - 19s 935us/step - loss: 2.4388 - dense_1_loss: 0.2216 - dense_2_loss: 1.1104 - dense_3_loss: 1.1068
Epoch 4/20
20000/20000 [==============================] - 19s 939us/step - loss: 2.4183 - dense_1_loss: 0.2097 - dense_2_loss: 1.1065 - dense_3_loss: 1.1021
Epoch 5/20
20000/20000 [==============================] - 19s 929us/step - loss: 2.4021 - dense_1_loss: 0.2018 - dense_2_loss: 1.1036 - dense_3_loss: 1.0966
Epoch 6/20
20000/20000 [==============================] - 19s 929us/step - loss: 2.3879 - dense_1_loss: 0.1966 - dense_2_loss: 1.1016 - dense_3_loss: 1.0897
Epoch 7/20
20000/20000 [==============================] - 19

In [26]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=32, epochs=20)

Epoch 1/20
20000/20000 [==============================] - 19s 934us/step - loss: 1.9330 - dense_1_loss: 0.1761 - dense_2_loss: 1.0816 - dense_3_loss: 0.6753
Epoch 2/20
20000/20000 [==============================] - 19s 932us/step - loss: 1.9106 - dense_1_loss: 0.1762 - dense_2_loss: 1.0808 - dense_3_loss: 0.6536
Epoch 3/20
20000/20000 [==============================] - 19s 955us/step - loss: 1.8908 - dense_1_loss: 0.1757 - dense_2_loss: 1.0807 - dense_3_loss: 0.6344
Epoch 4/20
20000/20000 [==============================] - 19s 949us/step - loss: 1.8744 - dense_1_loss: 0.1755 - dense_2_loss: 1.0800 - dense_3_loss: 0.6189
Epoch 5/20
20000/20000 [==============================] - 19s 953us/step - loss: 1.8591 - dense_1_loss: 0.1754 - dense_2_loss: 1.0796 - dense_3_loss: 0.6040
Epoch 6/20
20000/20000 [==============================] - 19s 950us/step - loss: 1.8475 - dense_1_loss: 0.1754 - dense_2_loss: 1.0802 - dense_3_loss: 0.5919
Epoch 7/20
20000/20000 [==============================] - 

In [27]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens))]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_denses = [Dense(n_tokens, activation='softmax')]
generator_outputs = [generator_denses[0](generator_lstm_out[0])]

In [28]:
generator_model_line0 = Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])
generator_model_line0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (1, None, 61)             0         
_________________________________________________________________
lstm_6 (LSTM)                [(1, None, 512), (1, 512) 1175552   
_________________________________________________________________
dense_6 (Dense)              (1, None, 61)             31293     
Total params: 1,206,845
Trainable params: 1,206,845
Non-trainable params: 0
_________________________________________________________________


In [29]:
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

In [36]:
generator_model_line0.reset_states()
line = [42]

def f():
    while line[-1] != 60:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
while line[-1] != 60:
    character, h, c = generator_model_line0.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[42, 42, 43, 44, 60]

In [31]:
generator_inputs_h = [None, Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,))]

generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))

generator_lstm_out[1], generator_lstm_h[1], generator_lstm_c[1] = generator_lstms[1](
    generator_inputs[1], initial_state=[generator_inputs_h[1], generator_inputs_c[1]]
)
generator_denses.append(Dense(n_tokens, activation='softmax'))
generator_outputs.append(generator_denses[1](generator_lstm_out[1]))

generator_model_line1 = Model(
    [generator_inputs[1], generator_inputs_h[1], generator_inputs_c[1]],
    [generator_outputs[1], generator_lstm_h[1], generator_lstm_c[1]]
)
generator_model_line1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (1, None, 61)        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 512)          0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(1, None, 512), (1, 1175552     input_11[0][0]                   
                                                                 input_9[0][0]                    
          

In [32]:
generator_lstms[1].set_weights(lstms[1].get_weights())
generator_denses[1].set_weights(denses[1].get_weights())

In [37]:
#generator_model_line1.reset_states()
#generator_line1_lstm.reset_states()
#generator_lstms[1].reset_states()
line = [0]
#h = np.array([np.zeros(256)])
#c = np.array([np.zeros(256)])

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_model_line1.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[0, 11, 10, 9, 60]